# Runner for Experiment 1


## Experiment

In [1]:
import pandas as pd
import matplotlib.pylab as o

from qiskit import assemble
from qiskit.transpiler import CouplingMap
from qiskit.test.mock.backends import FakeManhattan
from qiskit.transpiler.passes import DenseLayout, ExVF2Layout, \
                                     NoiseAdaptiveLayout, SabreLayout

from exp.custom_passmanager import custom_pass_manager
from exp.circuit_generators import graphstate_complete, graphstate_manhattan_ring

/Applications/anaconda3/envs/qiskit-terra-dev/lib/python3.8/site-packages/unittest2/compatibility.py:143: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  class ChainMap(collections.MutableMapping):


In [2]:
def exp1(circuit, layouter, backend, layouter_reps=100):
    passmanager = custom_pass_manager(backend, layouter, layout_reps=layouter_reps)

    times = {}
    count_ops_after_map = {}

    def callback(**kwargs):
        times[kwargs['pass_'].name()] = times.get(kwargs['pass_'].name(), 0) + kwargs['time']
        if kwargs['pass_'].name() == 'StochasticSwap':
            count_ops_after_map.update(kwargs['dag'].count_ops())
            
    
    transpiled = passmanager.run(circuit, callback=callback)

    needed_swaps = 0 if passmanager.property_set['is_swap_mapped'] else count_ops_after_map['swap']
    time = times[layouter.name()] / layouter_reps

    return time, needed_swaps

/Applications/anaconda3/envs/qiskit-terra-dev/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Execution

In [3]:
layouter_reps = 100
backend = FakeManhattan()
backend_properties = backend.properties()
coupling_map = CouplingMap(backend.configuration().coupling_map)

layout_type_dict = {
    'exvf2': ExVF2Layout(coupling_map), 
    'dense': DenseLayout(coupling_map, backend_properties), 
    'noise_adaptive': NoiseAdaptiveLayout(backend_properties),
    'sabre': SabreLayout(coupling_map, max_iterations=1)
}
circuit_type_dict = {
    'full': graphstate_complete(65),
    '12q_ring': graphstate_manhattan_ring(12),
    '12q_full': graphstate_complete(12)
}

In [10]:
df = pd.DataFrame(columns=["layouter", "circuit", "time", "swaps"])

# loop over layouters
for layouter_name, layouter in layout_type_dict.items():

    # loop over circuits
    for circuit_name, circuit in circuit_type_dict.items():

        time, swaps = exp1(circuit, layouter, backend, layouter_reps)
        result = {'layouter': layouter_name,
                  'circuit': circuit_name,
                  'time': time,
                  'swaps': swaps}
        df = df.append(result, ignore_index=True)

/Applications/anaconda3/envs/qiskit-terra-dev/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
print(df)

  layouter   circuit      time swaps
0    exvf2      full  0.000197  6875
1    exvf2  12q_ring  0.000125     0
2    exvf2  12q_full  0.726351   120
3    dense      full  0.002596  7228
4    dense  12q_ring  0.000565     7
5    dense  12q_full  0.000510    84
6    sabre      full  0.127225  6761
7    sabre  12q_ring  0.000320    33
8    sabre  12q_full  0.001559   107
